In [1]:
import pandas as pd
import sqlite3
import json
import matplotlib.pyplot as plt
import seaborn as sns

print("Libraries Imported Successfully")

Libraries Imported Successfully


In [6]:
# 1. Load CSV (Orders)
try:
    orders = pd.read_csv('/content/orders.csv')
    print(f"Orders loaded: {orders.shape}")
except FileNotFoundError:
    print("Error: orders.csv not found. Please upload it.")

# 2. Load JSON (Users)
try:
    users = pd.read_json('/content/users.json')
    print(f"Users loaded: {users.shape}")
except ValueError:
    with open('users.json', 'r') as f:
        data = json.load(f)
    users = pd.json_normalize(data)
    print(f"Users loaded (via json_normalize): {users.shape}")
except FileNotFoundError:
    print("Error: users.json not found. Please upload it.")

# 3. Load SQL (Restaurants)
try:
    # Create in-memory DB
    con = sqlite3.connect(':memory:')
    with open('/content/restaurants.sql', 'r') as f:
        sql_script = f.read()
    con.executescript(sql_script)

    # Extract data to DataFrame
    restaurants = pd.read_sql("SELECT * FROM restaurants", con)
    print(f"Restaurants loaded: {restaurants.shape}")
except Exception as e:
    print(f"Error loading SQL: {e}")

Orders loaded: (10000, 6)
Users loaded: (3000, 4)
Restaurants loaded: (500, 4)


In [7]:
# Merge Orders with Users (Left Join)
orders_users = pd.merge(orders, users, on='user_id', how='left')

# Merge Result with Restaurants (Left Join)
final_df = pd.merge(orders_users, restaurants, on='restaurant_id', how='left', suffixes=('', '_restaurant_master'))

# Convert Date Column
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)

print(f"Final Merged Dataset Shape: {final_df.shape}")
print(final_df.head())

Final Merged Dataset Shape: (10000, 12)
   order_id  user_id  restaurant_id order_date  total_amount  \
0         1     2508            450 2023-02-18        842.97   
1         2     2693            309 2023-01-18        546.68   
2         3     2084            107 2023-07-15        163.93   
3         4      319            224 2023-10-04       1155.97   
4         5     1064            293 2023-12-25       1321.91   

                  restaurant_name       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_restaurant_master  cuisine  rating  
0                    Restaurant_450  Mexican     3.2  
1                    Restaurant_309   Indian

In [8]:
# Q1
q1 = final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().idxmax()
print(f"Q1 (Highest Gold Revenue City): {q1}")

# Q2
q2 = final_df.groupby('cuisine')['total_amount'].mean().idxmax()
print(f"Q2 (Highest Avg Order Value Cuisine): {q2}")

# Q3
user_totals = final_df.groupby('user_id')['total_amount'].sum()
q3 = (user_totals > 1000).sum()
print(f"Q3 (Users with >1000 spend): {q3}")

# Q4
bins = [2.9, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']
final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels)
q4 = final_df.groupby('rating_range', observed=False)['total_amount'].sum().idxmax()
print(f"Q4 (Highest Revenue Rating Range): {q4}")

# Q5
q5 = final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].mean().idxmax()
print(f"Q5 (Highest Gold Avg Order Value City): {q5}")

# Q6
q6 = final_df.groupby('cuisine')['restaurant_id'].nunique().idxmin()
print(f"Q6 (Lowest Distinct Restaurants Cuisine): {q6}")

# Q7
q7_pct = (len(final_df[final_df['membership'] == 'Gold']) / len(final_df)) * 100
print(f"Q7 (Gold Order %): {round(q7_pct)}%")

# Q8
rest_stats = final_df.groupby('restaurant_name').agg(
    order_count=('order_id', 'count'),
    avg_value=('total_amount', 'mean')
)
q8 = rest_stats[rest_stats['order_count'] < 20].sort_values(by='avg_value', ascending=False).head(1).index[0]
print(f"Q8 (Highest AOV with <20 orders): {q8}")

# Q9
print("Q9 Analysis:")
options = [('Gold', 'Indian'), ('Gold', 'Italian'), ('Regular', 'Indian'), ('Regular', 'Chinese')]
best_option = None
max_revenue = -1
for membership, cuisine in options:
    rev = final_df[(final_df['membership'] == membership) & (final_df['cuisine'] == cuisine)]['total_amount'].sum()
    if rev > max_revenue:
        max_revenue = rev
        best_option = f"{membership} + {cuisine}"
print(f"Q9 Answer: {best_option}")

# Q10
final_df['quarter'] = final_df['order_date'].dt.quarter
quarter_map = {1: 'Q1', 2: 'Q2', 3: 'Q3', 4: 'Q4'}
q10_val = final_df.groupby('quarter')['total_amount'].sum().idxmax()
print(f"Q10 (Highest Revenue Quarter): {quarter_map[q10_val]}")

Q1 (Highest Gold Revenue City): Chennai
Q2 (Highest Avg Order Value Cuisine): Mexican
Q3 (Users with >1000 spend): 2544
Q4 (Highest Revenue Rating Range): 4.6 – 5.0
Q5 (Highest Gold Avg Order Value City): Chennai
Q6 (Lowest Distinct Restaurants Cuisine): Chinese
Q7 (Gold Order %): 50%
Q8 (Highest AOV with <20 orders): Hotel Dhaba Multicuisine
Q9 Analysis:
Q9 Answer: Gold + Italian
Q10 (Highest Revenue Quarter): Q3


In [9]:
# Q11
q11 = len(final_df[final_df['membership'] == 'Gold'])
print(f"Q11 (Total Gold Orders): {q11}")

# Q12
q12_val = final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()
print(f"Q12 (Hyderabad Revenue): {round(q12_val)}")

# Q13
q13 = final_df['user_id'].nunique()
print(f"Q13 (Distinct Users): {q13}")

# Q14
q14_val = final_df[final_df['membership'] == 'Gold']['total_amount'].mean()
print(f"Q14 (Gold Avg Order Value): {round(q14_val, 2)}")

# Q15
q15 = len(final_df[final_df['rating'] >= 4.5])
print(f"Q15 (High Rated Orders): {q15}")

# Q16
gold_df = final_df[final_df['membership'] == 'Gold']
top_gold_city = gold_df.groupby('city')['total_amount'].sum().idxmax()
q16 = len(gold_df[gold_df['city'] == top_gold_city])
print(f"Q16 (Orders in Top Gold City - {top_gold_city}): {q16}")

Q11 (Total Gold Orders): 4987
Q12 (Hyderabad Revenue): 1889367
Q13 (Distinct Users): 2883
Q14 (Gold Avg Order Value): 797.15
Q15 (High Rated Orders): 3374
Q16 (Orders in Top Gold City - Chennai): 1337
